# Konvertering fra File Geodatabase (GDB) til FlatGeobuf (FGB)
Denne notebooken viser hvordan du kan konvertere data fra Kartverkets N50-kartdata (gdb-format) til FlatGeobuf (fgb), et åpent og effektivt dataformat egnet for moderne utviklings- og webmiljøer.

## Hvorfor FlatGeobuf?
FlatGeobuf (FGB) gir flere fordeler sammenlignet med det tradisjonelle File Geodatabase-formatet (GDB):

- Åpent, lisensfritt og godt støttet i både Python, JavaScript og moderne GIS-verktøy.
- Én fil per datasett gjør deling, versjonskontroll og distribusjon enkelt.
- Rask innlesing, effektiv filstruktur og støtte for spatial indexering.
- Utviklervennlig og lett å bruke – basert på åpne formater uten lukkede lisenser

Denne notebooken gjør det enkelt å konvertere GDB-lag til FGB og kan utvides etter behov.

---

### Steg 1: Last ned kartdatene for N50
Naviger til Geonorge.no og finn kartdataene for N50, eller klikk på denne lenken - https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac

Her klikker du på valgmuligheten; "Last ned". N50 datasettet har nå blitt lagt til på Geonorges nedlastningside. Her velger du følgende valgmuligheter [... her må man utdype]

### Steg 2: Last ned nødvendige Python-pakker
For å utføre konverteringsprosessen trenger vi et par python-pakker. Disse har følgende bruksområder [... her må det utdypes]

In [1]:
# Installer nødvendige pakker
import geopandas as gpd
import fiona
from pathlib import Path

### Steg 3: List ut lag fra N50 datasett
Vi benytter funksjonen `list_gdb_layers` til å liste ut alle de tilgjengelige lagene som følger med N50 datasettet.

In [2]:
def list_gdb_layers(gdb_path):
    """
    List alle tilgjengelige lag i en GDB-fil.
    
    Args:
        gdb_path (str): Path til GDB-filen.
        
    Returns:
        list: Liste over lagnavn i GDB-filen.
    """
    try:
        layers = fiona.listlayers(str(gdb_path))
        print("\nTilgjengelige lag i GDB:")
        for idx, layer in enumerate(layers, 1):
            print(f"{idx}. {layer}")
        return layers
    except Exception as e:
        print(f"Feil ved listing av lag: {e}")
        return []

In [ ]:
# Sett path til GDB-mappen
gdb_path = Path("data/Basisdata_0000_Norge_25833_N50Kartdata_FGDB.gdb")

if not gdb_path.exists():
    print(f"Feil: Fant ikke GDB-fil på {gdb_path}")
else:
    # List ut alle lag i GDB-filen
    layers = list_gdb_layers(gdb_path)
    print(f"\nTotalt antall lag funnet: {len(layers)}")


### Steg 4: Utfør konverteringsoppgave
Vi benytter funksjonen `convert_layer_to_fgb` til å konvertere et gdb-lag til en fgb-fil. Her gjennbruker vi `gdb_path` fra tidligere, vi velger hvilket lag vi ønsker å konvertere med `layer_name`, og velger at den ferdige fgb-filen lander i en `output` mappe.

In [4]:
def convert_layer_to_fgb(gdb_path: Path, layer_name: str, output_dir: Path) -> Path:
    """
    Konverter et enkelt lag fra GDB til FlatGeobuf-format.
    
    Args:
        gdb_path (Path): Path til GDB-filen.
        layer_name (str): Navn på laget som skal konverteres.
        output_dir (Path): Mappe for lagring av resultatet.
        
    Returns:
        Path: Path til opprettet FlatGeobuf-fil.
    """
    try:
        output_dir.mkdir(parents=True, exist_ok=True)
        output_file = output_dir / f"{layer_name}.fgb"
        
        print(f"\nKonverterer lag: {layer_name}")
        print(f"Leser fra: {gdb_path}")
        print(f"Skriver til: {output_file}")
        
        print(f"\nLeser fra GDB... For store lag kan dette ta litt tid.")
        
        gdf = gpd.read_file(gdb_path, layer=layer_name)
        
        # Vis info om laget
        print(f"\nLaginformasjon:")
        print(f"Antall objekter: {len(gdf)}")
        print(f"Geometri-type: {gdf.geometry.type.iloc[0]}")
        print(f"Koordinatreferansesystem (CRS): {gdf.crs}")

        print(f"\nSkriver til FlatGeobuf-format... For store lag kan dette også ta litt tid.")

        gdf.to_file(output_file, driver="FlatGeobuf")
        
        print(f"\nKonvertering fullført!")
        return output_file
        
    except Exception as e:
        print(f"Feil under konvertering av lag: {e}")
        return None


In [ ]:
# Velg lag som skal konverteres (juster etter behov)
layer_name = "N50_Arealdekke_omrade"

# Sett path til output mappen
output_dir = Path("output")

# Konverter valgt lag til FGB
output_file = convert_layer_to_fgb(gdb_path, layer_name, output_dir)


## Videre bruk

Den konverterte `.fgb`-filen kan nå brukes direkte i f.eks. QGIS, web-apper (MapLibre GL, Deck.gl, etc.), eller enkelt leses inn i Python, JavaScript, eller andre miljøer med støtte for FlatGeobuf.

Dette gjør det lettere for utviklere å jobbe med åpne, norske kartdata!
